In [ ]:
import glob
import os.path
import datetime
from datetime import timedelta
import time
from subprocess import call
import subprocess
import urllib
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy import interpolate
from tol_colors import tol_cmap
import warnings

warnings.simplefilter("ignore")

plt.rcParams['figure.figsize'] = [24, 12]

interp_method = 'linear'#'nearest'
pps_user = #PPS User name
pps_pwd = #PPS password
data_dir = #Your data directory. Will need subdirectories for each sensor.
tropics_uid = #NASA Earthdata username
tropics_pass = #NASA Earthdata password

In [ ]:
#Download latest tcvitals
utctime = datetime.datetime.utcnow()
url = 'http://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.%4.4i%2.2i%2.2i%2.2i/atmos/gfs.t%2.2iz.syndata.tcvitals.tm00'%(utctime.year,utctime.month,utctime.day,(utctime.hour//6)*6,(utctime.hour//6)*6)
#call(['curl',url])
try:
    #f = open(url.split('/')[-1])
    f = urllib.request.urlopen(url)
    tcvitals = f.readlines()
    f.close()
except:
    try:
        utctime = utctime-datetime.timedelta(hours=6)
        url = 'http://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.%4.4i%2.2i%2.2i/%2.2i/atmos/gfs.t%2.2iz.syndata.tcvitals.tm00'%(utctime.year,utctime.month,utctime.day,(utctime.hour//6)*6,(utctime.hour//6)*6)
        #call(['curl',url])
        #f = open(url.split('/')[-1])
        f = urllib.request.urlopen(url)
        tcvitals = f.readlines()
        f.close()
    except:
        print("couldn't access nomads")
        tcvitals=[]
print(tcvitals)


#call(['curl','http://ftp.nhc.noaa.gov/atcf/com/tcvitals'])
#f=open('tcvitals')
try:
    f = urllib.request.urlopen('http://ftp.nhc.noaa.gov/atcf/com/tcvitals')
    tcvitals_nhc = f.readlines()
    f.close()
    tcvitals = tcvitals_nhc+tcvitals
except:
    print("couldn't access nhc")
    
#remove duplicate lines
tcvitals_new = []
for line in tcvitals:
    match=0
    for line2 in tcvitals_new:
        if(line2.split()[1] == line.split()[1]): match = 1
        #if(line[0:43] == line2[0:43]): match=1
        
    if(match == 0): tcvitals_new.append(line.replace(b':',b''))
#for line in tcvitals_new:
#	print(line[0:43])
tcvitals = tcvitals_new
#tcvitals.append(b'NHC 90L INVEST    20230923 1200 155N 0365W 999')
ax =plt.subplot(1,1,1,projection = ccrs. PlateCarree(), extent = [-180,180,-60,60])
for line in tcvitals:
    print(line)#.split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    ax.plot(tc_lon, tc_lat, 'ro', transform=ccrs.Geodetic())
    ax.text(tc_lon, tc_lat, tc_name,fontsize=15)

#Plot TC locations on map
ax.coastlines()
plt.show()

    
#override with manual location
#tcvitals = [b'NHC  99L CONUS     20240306 1200 380N 0800W 999']

In [ ]:
#Download last 12 hours of MHS data
flist = subprocess.check_output(['curl','-k', '-4','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/MHS/'],text=True)
#print(flist)
flist = flist.split('\n')
for fitem in flist[0:-1]:
    #print(fitem)
    file = fitem.split()[-1]
    #print(file)#,fsize)
    #only run if file is newer than end time from last 1C production file
    if((file[0:2] == '1C')):
        fsize = int(fitem.split()[4])
        fname = file.split('.')
        this_datetime = datetime.datetime(int(fname[4][0:4]),int(fname[4][4:6]),int(fname[4][6:8]),int(fname[4][18:20]),int(fname[4][20:22]),int(fname[4][22:24]))
        if(this_datetime > (datetime.datetime.utcnow() - timedelta(hours=6))):
            dname = (f'{data_dir}/MHS/{file}')
            dsize=-1
            if(os.path.isfile(dname)): dsize = os.path.getsize(dname)
            print(dname,dsize,fsize)
            if(fsize != dsize): call(['curl','-4','-k','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/MHS/'+file,'-o',dname])

#clean MHS directory
files = glob.glob(f'{data_dir}/MHS/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400): os.remove(file)

In [ ]:
#Plot MHS RGB
import warnings
warnings.filterwarnings('ignore')
rs = 7 #region size/2 in degrees

proj = ccrs.PlateCarree(central_longitude=180) 
plt.rcParams['figure.figsize'] = [12, 6]
for line in tcvitals:
    #print(str(line).split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    print(tc_id,tc_name,tc_time, tc_lat, tc_lon)  
    fix_time = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(tc_time.year,tc_time.month,tc_time.day,tc_time.hour,tc_time.minute)
    #if(tc_id[2] != 'L'):continue
    
    found_overpass = False
    mindist=9999.
    file_list = glob.glob(f'{data_dir}/MHS/1C.*')
    file_list.sort()
    for file in file_list:
        #print(file)
        infile = file.split('.')
        satid = infile[1]
        
        overpass_time = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][10:12]),int(infile[4][12:14]))
        
        if(abs(tc_time-overpass_time) < datetime.timedelta(hours=6)):
            
            h5id = Dataset(file)
            s1lat = np.array(h5id.groups['S1'].variables['Latitude'])
            s1lon = np.array(h5id.groups['S1'].variables['Longitude'])
            s1lon_unfolded = np.copy(s1lon)
            s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360.
            #print(s1lat.shape)
            dist = 6371.*np.arccos(np.sin(np.deg2rad(s1lat[:,45]))*np.sin(np.deg2rad(tc_lat)) + np.cos(np.deg2rad(s1lat[:,45]))*np.cos(np.deg2rad(tc_lat))*np.cos(np.deg2rad(s1lon[:,45])-np.deg2rad(tc_lon)))
            if(np.min(dist) > 600): 
                h5id.close()
                continue
            
            print(satid,abs(overpass_time-tc_time),np.min(dist))
            if(np.min(dist) < mindist):
                mindist = np.min(dist)
                mintime = np.argmin(dist)
            #print(np.unravel_index(mintime,s1lat.shape))
            s1hour = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Hour'])
            s1min = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Minute'])
            s1sec = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Second'])
            s1mon = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Month'])
            s1dom = np.array(h5id.groups['S1'].groups['ScanTime'].variables['DayOfMonth'])
            s1year = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Year'])
            try:
                mintimestamp = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(s1year[mintime],s1mon[mintime],s1dom[mintime],s1hour[mintime],s1min[mintime])
            except:
                mintimestamp = 'N/A'
            #print(mintimestamp)
            #    ds = string(s1dom[mintime],format='(I2.2)')
            #    fname = outdir+ys+'/'+ms+'/'+ds+'/'+id+'.'+strmid(mintimestamp,9,4)+'.'+satid+'.MHS.png'
            s1Tc = np.array(h5id.groups['S1'].variables['Tc'])
            #print(s1Tc.shape)
            h5id.close()
            
            #find min and max scan for plotting
            a = np.where((np.abs(s1lon-tc_lon) < 7) & (np.abs(s1lat-tc_lat) < 7))
            #print(len(a[0]))
            
            if(len(a[0]) > 0):
                s1min = np.min(a[0])#-1
                s1max = np.max(a[0])#+1
            
                s1lon = s1lon[s1min:s1max,:]
                s1lat = s1lat[s1min:s1max,:]
                s1Tc = s1Tc[s1min:s1max,:,:]
                
                s1lon_unfolded = np.copy(s1lon)
                s1lon_unfolded[s1lon < 0] += 360
            
            #unfold longitude
            #if(tc_lon < -10):
            #    s1lon = s1lon_unfolded[s1min:s1max,:]
            #Plot native resolution Tbs
            s1ch = ['89','157','183$\pm$1','183$\pm$3','190']
            for c in range(0,s1Tc.shape[2]):
                m = plt.subplot(2,s1Tc.shape[2],c+1, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=100,vmax=300, transform = ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c])
            plt.show()
            #create lat/lon grid for interpolation
            gridlats = np.linspace(tc_lat-rs,tc_lat+rs,num=rs*40+1)
            gridlons = np.linspace(tc_lon-rs,tc_lon+rs,num=rs*40+1)
            #gridlons[gridlons < -180] = gridlons[gridlons < -180] + 360
            #gridlons[gridlons > 180] = gridlons[gridlons > 180] - 360
            s1lon_unfolded = np.copy(s1lon)
            if(np.min(gridlons) < -180):
                s1lon_unfolded[s1lon > 0] = s1lon[s1lon > 0]-360
            elif(np.max(gridlons) > 180):
                s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360
            print(np.min(s1lon_unfolded),np.max(s1lon_unfolded))
            
            xy = np.zeros((s1lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s1lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s1lat)
            z = np.ndarray.flatten(s1Tc[:,:,1])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,4])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,3])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            maxt = 290#np.max(s1Tc[:,:,1:4])
            mint = 140#np.min(s1Tc[:,:,1:4])
            if(maxt > 300): maxt = 300
            if(mint < 50): mint = 50
            idata[:,:,0] = (maxt-rgrid)/(maxt-mint)
            idata[:,:,1] = (maxt-ggrid)/(maxt-mint)
            idata[:,:,2] = (maxt-bgrid)/(maxt-mint)
            idata[idata < 0] = 0
            idata[idata > 1] = 1
            #idata[idata > 255] = 255
            #idata = idata/255.
            
            #m = plt.subplot(2,s1Tc.shape[2],1, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,rgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],2, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,ggrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],3, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,bgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            #plt.show()
            m = plt.subplot(1,1,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '183 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="white", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            m.coastlines('50m', linewidth=0.8,color='white')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'\n '+satid+' MHS Overpass time: '+mintimestamp)
            plt.show()

    

In [ ]:
#Download last 12 hours of ATMS data
flist = subprocess.check_output(['curl','-k', '-4','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/ATMS/'],text=True)
#print(flist)
flist = flist.split('\n')
for fitem in flist[0:-1]:
    #print(fitem)
    file = fitem.split()[-1]
    #print(file)#,fsize)
    #only run if file is newer than end time from last 1C production file
    if((file[0:2] == '1C')):
        fsize = int(fitem.split()[4])
        fname = file.split('.')
        this_datetime = datetime.datetime(int(fname[4][0:4]),int(fname[4][4:6]),int(fname[4][6:8]),int(fname[4][18:20]),int(fname[4][20:22]),int(fname[4][22:24]))
        if(this_datetime > (datetime.datetime.utcnow() - timedelta(hours=12))):
            dname = (f'{data_dir}/ATMS/{file}')
            dsize=-1
            if(os.path.isfile(dname)): dsize = os.path.getsize(dname)
            print(dname,dsize,fsize)
            if(fsize != dsize): call(['curl','-4','-k','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/ATMS/'+file,'-o',dname])

#clean MHS directory
files = glob.glob(f'{data_dir}/ATMS/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400): os.remove(file)

In [ ]:
#Plot ATMS RGB
proj = ccrs.PlateCarree(central_longitude=180) 
plt.rcParams['figure.figsize'] = [12, 6]
rs=7
for line in tcvitals:
    #print(str(line).split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    #if(tc_id[2] != 'L'):continue
    print(tc_id,tc_name,tc_time, tc_lat, tc_lon)  
    fix_time = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(tc_time.year,tc_time.month,tc_time.day,tc_time.hour,tc_time.minute)
            
    
    found_overpass = False
    mindist=9999.
    file_list = glob.glob(f'{data_dir}/ATMS/1C.*')
    file_list.sort()
    for file in file_list:
        #print(file)
        infile = file.split('.')
        satid = infile[1]
        
        overpass_time = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][10:12]),int(infile[4][12:14]))
        
        if(abs(tc_time-overpass_time) < datetime.timedelta(hours=12)):
            
            h5id = Dataset(file)
            #print(h5id.groups)
            #stop
            s4lat = np.array(h5id.groups['S4'].variables['Latitude'])
            s4lon = np.array(h5id.groups['S4'].variables['Longitude'])
            #s1lon_unfolded = s1lon
            #s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360.
            #print(s1lat.shape)
            dist = 6371.*np.arccos(np.sin(np.deg2rad(s4lat[:,45]))*np.sin(np.deg2rad(tc_lat)) + np.cos(np.deg2rad(s4lat[:,45]))*np.cos(np.deg2rad(tc_lat))*np.cos(np.deg2rad(s4lon[:,45])-np.deg2rad(tc_lon)))
            if(np.min(dist) > 600): 
                h5id.close()
                continue
            
            print(satid,abs(overpass_time-tc_time),np.min(dist))
            if(np.min(dist) < mindist):
                mindist = np.min(dist)
                mintime = np.argmin(dist)
            #print(np.unravel_index(mintime,s1lat.shape))
            s4hour = np.array(h5id.groups['S4'].groups['ScanTime'].variables['Hour'])
            s4min = np.array(h5id.groups['S4'].groups['ScanTime'].variables['Minute'])
            s4sec = np.array(h5id.groups['S4'].groups['ScanTime'].variables['Second'])
            s4mon = np.array(h5id.groups['S4'].groups['ScanTime'].variables['Month'])
            s4dom = np.array(h5id.groups['S4'].groups['ScanTime'].variables['DayOfMonth'])
            s4year = np.array(h5id.groups['S4'].groups['ScanTime'].variables['Year'])

            mintimestamp = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(s4year[mintime],s4mon[mintime],s4dom[mintime],s4hour[mintime],s4min[mintime])
            
            s1lat = np.array(h5id.groups['S1'].variables['Latitude'])
            s1lon = np.array(h5id.groups['S1'].variables['Longitude'])
            s1Tc = np.array(h5id.groups['S1'].variables['Tc'])
            print(s1Tc.shape)
            
            s2lat = np.array(h5id.groups['S2'].variables['Latitude'])
            s2lon = np.array(h5id.groups['S2'].variables['Longitude'])
            s2Tc = np.array(h5id.groups['S2'].variables['Tc'])
            print(s2Tc.shape)
            s3lat = np.array(h5id.groups['S3'].variables['Latitude'])
            s3lon = np.array(h5id.groups['S3'].variables['Longitude'])
            s3Tc = np.array(h5id.groups['S3'].variables['Tc'])
            print(s3Tc.shape)
            s4Tc = np.array(h5id.groups['S4'].variables['Tc'])
            #print(s1Tc.shape)
            print(s4Tc.shape)
            h5id.close()
            
            #find min and max scan for plotting
            a = np.where((np.abs(s4lon-tc_lon) < 7) & (np.abs(s4lat-tc_lat) < 7))
            #print(len(a[0]))
            
            if(len(a[0]) > 0):
                s4min = np.min(a[0])#-1
                s4max = np.max(a[0])#+1
            
                s4lon = s4lon[s4min:s4max,:]
                s4lat = s4lat[s4min:s4max,:]
                s4Tc = s4Tc[s4min:s4max,:,:]
                
                s1lon = s1lon[s4min:s4max,:]
                s1lat = s1lat[s4min:s4max,:]
                s1Tc = s1Tc[s4min:s4max,:,:]
                
                s2lon = s2lon[s4min:s4max,:]
                s2lat = s2lat[s4min:s4max,:]
                s2Tc = s2Tc[s4min:s4max,:,:]
                
                s3lon = s3lon[s4min:s4max,:]
                s3lat = s3lat[s4min:s4max,:]
                s3Tc = s3Tc[s4min:s4max,:,:]
                
                s1lon_unfolded = np.copy(s1lon)
                s1lon_unfolded[s1lon < 0] += 360
                s2lon_unfolded = np.copy(s2lon)
                s2lon_unfolded[s2lon < 0] += 360
                s3lon_unfolded = np.copy(s3lon)
                s3lon_unfolded[s3lon < 0] += 360
                s4lon_unfolded = np.copy(s4lon)
                s4lon_unfolded[s4lon < 0] += 360
            
            #Plot native resolution Tbs
            m = plt.subplot(2,5,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,0],cmap=tol_cmap(colormap='rainbow_WhBr'),
                              vmin=100,vmax=300,transform=ccrs.PlateCarree())
            plt.colorbar(cm,orientation='horizontal')
            plt.title('23.8 GHz')
            
            m = plt.subplot(2,5,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s2lon_unfolded,s2lat,s2Tc[:,:,0],cmap=tol_cmap(colormap='rainbow_WhBr'),
                              vmin=100,vmax=300,transform=ccrs.PlateCarree())
            plt.colorbar(cm,orientation='horizontal')
            plt.title('31.4 GHz')
            
            m = plt.subplot(2,5,3, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s3lon_unfolded,s3lat,s3Tc[:,:,0],cmap=tol_cmap(colormap='rainbow_WhBr'),
                              vmin=100,vmax=300,transform=ccrs.PlateCarree())
            plt.colorbar(cm,orientation='horizontal')
            plt.title('88.2 GHz')
            
            m = plt.subplot(2,5,4, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s4lon_unfolded,s4lat,s4Tc[:,:,0],cmap=tol_cmap(colormap='rainbow_WhBr'),
                              vmin=100,vmax=300,transform=ccrs.PlateCarree())
            plt.colorbar(cm,orientation='horizontal')
            plt.title('165.5 GHz')
                
            s4ch = ['183$\pm$7','183$\pm$4.5','183$\pm$3','183$\pm$1.8','183$\pm$1']
            for c in range(1,s4Tc.shape[2]):
                m = plt.subplot(2,5,c+5, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s4lon_unfolded,s4lat,s4Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=100,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s4ch[c-1])
            plt.show()
            
            #create lat/lon grid for interpolation
            gridlats = np.linspace(tc_lat-rs,tc_lat+rs,num=rs*40+1)
            gridlons = np.linspace(tc_lon-rs,tc_lon+rs,num=rs*40+1)
            #gridlons[gridlons < -180] = gridlons[gridlons < -180] + 360
            #gridlons[gridlons > 180] = gridlons[gridlons > 180] - 360
            
            s2lon_unfolded = np.copy(s2lon)
            s3lon_unfolded = np.copy(s3lon)
            s4lon_unfolded = np.copy(s4lon)
            if(np.min(gridlons) < -180):
                s2lon_unfolded[s2lon > 0] = s2lon[s2lon > 0]-360
                s3lon_unfolded[s3lon > 0] = s3lon[s3lon > 0]-360
                s4lon_unfolded[s4lon > 0] = s4lon[s4lon > 0]-360
            elif(np.max(gridlons) > 180):
                s2lon_unfolded[s2lon < 0] = s2lon[s2lon < 0]+360
                s3lon_unfolded[s3lon < 0] = s3lon[s3lon < 0]+360
                s4lon_unfolded[s4lon < 0] = s4lon[s4lon < 0]+360

            xy = np.zeros((s4lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s4lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s4lat)
            z = np.ndarray.flatten(s4Tc[:,:,1])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s4Tc[:,:,3])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s4Tc[:,:,5])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            maxt = 290#np.max(s4Tc)
            mint = 140#np.min(s4Tc)
            idata[:,:,0] = (maxt-rgrid)/(maxt-mint)
            idata[:,:,1] = (maxt-ggrid)/(maxt-mint)
            idata[:,:,2] = (maxt-bgrid)/(maxt-mint)
            idata[idata < 0] = 0
            idata[idata > 1] = 1
            #idata = idata/255.
            
            #m = plt.subplot(2,s1Tc.shape[2],1, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,rgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],2, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,ggrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],3, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,bgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            #plt.show()
            m = plt.subplot(1,2,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '183 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="white", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            m.coastlines('50m', linewidth=0.8,color='white')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' ATMS Overpass time: '+mintimestamp)
            
            xy = np.zeros((s2lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s2lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s2lat)
            z = np.ndarray.flatten(s2Tc[:,:,0])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            xy = np.zeros((s3lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s3lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s3lat)
            z = np.ndarray.flatten(s3Tc[:,:,0])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            xy = np.zeros((s4lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s4lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s4lat)
            z = np.ndarray.flatten(s4Tc[:,:,0])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.
            
            m = plt.subplot(1,2,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '31 + 88 + 165 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="white", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            m.coastlines('50m', linewidth=0.8,color='white')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+';\n '+satid+' ATMS Overpass time: '+mintimestamp)
          
            plt.show()

    

In [ ]:
#Download last 12 hours of AMSR2 data
flist = subprocess.check_output(['curl','-k', '-4','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/AMSR2/'],text=True)
#print(flist)
flist = flist.split('\n')
for fitem in flist[0:-1]:
    #print(fitem)
    file = fitem.split()[-1]
    #print(file)#,fsize)
    #only run if file is newer than end time from last 1C production file
    if((file[0:2] == '1C')):
        fsize = int(fitem.split()[4])
        fname = file.split('.')
        this_datetime = datetime.datetime(int(fname[4][0:4]),int(fname[4][4:6]),int(fname[4][6:8]),int(fname[4][18:20]),int(fname[4][20:22]),int(fname[4][22:24]))
        if(this_datetime > (datetime.datetime.utcnow() - timedelta(hours=12))):
            dname = (f'{data_dir}/AMSR2/{file}')
            dsize=-1
            if(os.path.isfile(dname)): dsize = os.path.getsize(dname)
            print(dname,dsize,fsize)
            if(fsize != dsize): call(['curl','-4','-k','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/AMSR2/'+file,'-o',dname])

#clean MHS directory
files = glob.glob(f'{data_dir}/AMSR2/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400): os.remove(file)

In [ ]:
#Plot AMSR2 RGB
proj = ccrs.PlateCarree(central_longitude=180) 
plt.rcParams['figure.figsize'] = [16, 8]
rs=7
for line in tcvitals:
    #print(str(line).split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    #if(tc_id[2] != 'L'):continue
    print(tc_id,tc_name,tc_time, tc_lat, tc_lon)  
    fix_time = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(tc_time.year,tc_time.month,tc_time.day,tc_time.hour,tc_time.minute)
            
    
    found_overpass = False
    mindist=9999.
    file_list = glob.glob(f'{data_dir}/AMSR2/1C.*')
    file_list.sort()
    for file in file_list:
        #print(file)
        infile = file.split('.')
        satid = infile[1]
        
        overpass_time = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][10:12]),int(infile[4][12:14]))
        
        if(abs(tc_time-overpass_time) < datetime.timedelta(hours=12)):
            
            h5id = Dataset(file)
            #print(h5id.groups)
            #stop
            s1lat = np.array(h5id.groups['S1'].variables['Latitude'])
            s1lon = np.array(h5id.groups['S1'].variables['Longitude'])
            #s1lon_unfolded = s1lon
            #s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360.
            #print(s1lat.shape)
            
            dist = 6371.*np.arccos(np.sin(np.deg2rad(s1lat[:,121]))*np.sin(np.deg2rad(tc_lat)) + np.cos(np.deg2rad(s1lat[:,121]))*np.cos(np.deg2rad(tc_lat))*np.cos(np.deg2rad(s1lon[:,121])-np.deg2rad(tc_lon)))
            if(np.min(dist) > 800): 
                h5id.close()
                continue
            
            print(satid,abs(overpass_time-tc_time),np.min(dist))
            if(np.min(dist) < mindist):
                mindist = np.min(dist)
                mintime = np.argmin(dist)
            #print(np.unravel_index(mintime,s1lat.shape))
            s1hour = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Hour'])
            s1min = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Minute'])
            s1sec = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Second'])
            s1mon = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Month'])
            s1dom = np.array(h5id.groups['S1'].groups['ScanTime'].variables['DayOfMonth'])
            s1year = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Year'])

            mintimestamp = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(s1year[mintime],s1mon[mintime],s1dom[mintime],s1hour[mintime],s1min[mintime])
            
            s1Tc = np.array(h5id.groups['S1'].variables['Tc'])
            
            s2lat = np.array(h5id.groups['S2'].variables['Latitude'])
            s2lon = np.array(h5id.groups['S2'].variables['Longitude'])
            s2Tc = np.array(h5id.groups['S2'].variables['Tc'])
            
            s3lat = np.array(h5id.groups['S3'].variables['Latitude'])
            s3lon = np.array(h5id.groups['S3'].variables['Longitude'])
            s3Tc = np.array(h5id.groups['S3'].variables['Tc'])
            
            s4lat = np.array(h5id.groups['S4'].variables['Latitude'])
            s4lon = np.array(h5id.groups['S4'].variables['Longitude'])
            s4Tc = np.array(h5id.groups['S4'].variables['Tc'])
            
            s5lat = np.array(h5id.groups['S5'].variables['Latitude'])
            s5lon = np.array(h5id.groups['S5'].variables['Longitude'])
            s5Tc = np.array(h5id.groups['S5'].variables['Tc'])
            
            s6lat = np.array(h5id.groups['S6'].variables['Latitude'])
            s6lon = np.array(h5id.groups['S6'].variables['Longitude'])
            s6Tc = np.array(h5id.groups['S6'].variables['Tc'])
            #print(s1Tc.shape)
            h5id.close()
            
            #find min and max scan for plotting
            a = np.where((np.abs(s1lon-tc_lon) < 6) & (np.abs(s1lat-tc_lat) < 6))
            #print(len(a[0]))
            
            if(len(a[0]) > 0):
                s1min = np.min(a[0])#-1
                s1max = np.max(a[0])#+1
                
                s1lon = s1lon[s1min:s1max,:]
                s1lat = s1lat[s1min:s1max,:]
                s1Tc = s1Tc[s1min:s1max,:,:]
                
                s2lon = s2lon[s1min:s1max,:]
                s2lat = s2lat[s1min:s1max,:]
                s2Tc = s2Tc[s1min:s1max,:,:]
                
                s3lon = s3lon[s1min:s1max,:]
                s3lat = s3lat[s1min:s1max,:]
                s3Tc = s3Tc[s1min:s1max,:,:]
                
                s4lon = s4lon[s1min:s1max,:]
                s4lat = s4lat[s1min:s1max,:]
                s4Tc = s4Tc[s1min:s1max,:,:]
                
                s5lon = s5lon[s1min:s1max,:]
                s5lat = s5lat[s1min:s1max,:]
                s5Tc = s5Tc[s1min:s1max,:,:]
                
                s6lon = s6lon[s1min:s1max,:]
                s6lat = s6lat[s1min:s1max,:]
                s6Tc = s6Tc[s1min:s1max,:,:]
                
                s1lon_unfolded = np.copy(s1lon)
                s1lon_unfolded[s1lon < 0] += 360
                s2lon_unfolded = np.copy(s2lon)
                s2lon_unfolded[s2lon < 0] += 360
                s3lon_unfolded = np.copy(s3lon)
                s3lon_unfolded[s3lon < 0] += 360
                s4lon_unfolded = np.copy(s4lon)
                s4lon_unfolded[s4lon < 0] += 360
                s5lon_unfolded = np.copy(s5lon)
                s5lon_unfolded[s5lon < 0] += 360
                s6lon_unfolded = np.copy(s6lon)
                s6lon_unfolded[s6lon < 0] += 360
                
                
            
            #Plot native resolution Tbs
            s1ch = ['10V','10H','18V','18H','23V','23H','36V','36H','89V','89H','89V','89H']
            for c in range(0,2):
                m = plt.subplot(2,6,1+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c])
            
                m = plt.subplot(2,6,2+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s2lon_unfolded,s2lat,s2Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c+2])
                
                m = plt.subplot(2,6,3+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s3lon_unfolded,s3lat,s3Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c+4])
            
                m = plt.subplot(2,6,4+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s4lon_unfolded,s4lat,s4Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c+6])
            
                m = plt.subplot(2,6,5+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s5lon_unfolded,s5lat,s5Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr')
                                  ,vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c+8])
            
                m = plt.subplot(2,6,6+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s6lon_unfolded,s6lat,s6Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c+10])
            plt.show()
            
            
            
            #create lat/lon grid for interpolation
            gridlats = np.linspace(tc_lat-rs,tc_lat+rs,num=rs*40+1)
            gridlons = np.linspace(tc_lon-rs,tc_lon+rs,num=rs*40+1)
            #gridlons[gridlons < -180] = gridlons[gridlons < -180] + 360
            #gridlons[gridlons > 180] = gridlons[gridlons > 180] - 360
            
            s4lon_unfolded = np.copy(s4lon)
            if(np.min(gridlons) < -180):
                s4lon_unfolded[s4lon > 0] = s4lon[s4lon > 0]-360
            elif(np.max(gridlons) > 180):
                s4lon_unfolded[s4lon < 0] = s4lon[s4lon < 0]+360

            xy = np.zeros((s4lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s4lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s4lat)
            z = np.ndarray.flatten(s2Tc[:,:,0])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s2Tc[:,:,1])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,1])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.
            
            #m = plt.subplot(2,s1Tc.shape[2],1, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,rgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],2, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,ggrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],3, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,bgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            #plt.show()
            m = plt.subplot(1,2,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '10 + 18 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.xlabels_top = False
            #m.coastlines('50m', linewidth=0.8,color='black')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' AMSR2 Overpass time: '+mintimestamp)
            
            s7lat = np.zeros((s5lat.shape[0]*2,s5lat.shape[1]),dtype=np.float32)
            s7lon = np.zeros((s5lat.shape[0]*2,s5lat.shape[1]),dtype=np.float32)
            s7Tc = np.zeros((s5lat.shape[0]*2,s5lat.shape[1],2),dtype=np.float32)
            
            s7lat[::2,:] = s5lat
            s7lon[::2,:] = s5lon
            s7Tc[::2,:,:] = s5Tc
            s7lat[1::2,:] = s6lat
            s7lon[1::2,:] = s6lon
            s7Tc[1::2,:,:] = s6Tc
            
            s7lon_unfolded = np.copy(s7lon)
            if(np.min(gridlons) < -180):
                s7lon_unfolded[s7lon > 0] = s7lon[s7lon > 0]-360
            elif(np.max(gridlons) > 180):
                s7lon_unfolded[s7lon < 0] = s7lon[s7lon < 0]+360
            
            xy = np.zeros((s7lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s7lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s7lat)
            z = np.ndarray.flatten(s7Tc[:,:,0])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            z = np.ndarray.flatten(s7Tc[:,:,1])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            xy = np.zeros((s4lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s4lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s4lat)
            z = np.ndarray.flatten(s4Tc[:,:,1])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.
            
            m = plt.subplot(1,2,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            #m.scatter(s5lon,s5lat,c=s5Tc[:,:,1],s=1,vmin=100,vmax=300,cmap=plt.cm.nipy_spectral)
            #m.scatter(s6lon,s6lat,c=s6Tc[:,:,1],s=1,vmin=100,vmax=300,cmap=plt.cm.nipy_spectral)
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '37 + 89 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            #m.coastlines('50m', linewidth=0.8,color='black')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+';\n '+satid+' AMSR2 Overpass time: '+mintimestamp)
          
            plt.show()
            mindist = 9999
            found_overpass = False
    

In [ ]:
#Download last 12 hours of SSMIS data
flist = subprocess.check_output(['curl','-k', '-4','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/SSMIS/'],text=True)
#print(flist)
flist = flist.split('\n')
for fitem in flist[0:-1]:
    #print(fitem)
    file = fitem.split()[-1]
    #print(file)#,fsize)
    #only run if file is newer than end time from last 1C production file
    if((file[0:2] == '1C')):
        fsize = int(fitem.split()[4])
        fname = file.split('.')
        this_datetime = datetime.datetime(int(fname[4][0:4]),int(fname[4][4:6]),int(fname[4][6:8]),int(fname[4][18:20]),int(fname[4][20:22]),int(fname[4][22:24]))
        if(this_datetime > (datetime.datetime.utcnow() - timedelta(hours=12))):
            dname = (f'{data_dir}/SSMIS/{file}')
            dsize=-1
            if(os.path.isfile(dname)): dsize = os.path.getsize(dname)
            print(dname,dsize,fsize)
            if(fsize != dsize): call(['curl','-4','-k','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/SSMIS/'+file,'-o',dname])

#clean MHS directory
files = glob.glob(f'{data_dir}/SSMIS/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400): os.remove(file)

In [ ]:
#Plot SSMIS RGB
proj = ccrs.PlateCarree(central_longitude=180) 
rs=7
plt.rcParams['figure.figsize'] = [16, 8]
for line in tcvitals:
    #print(str(line).split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    #if(tc_id[2] != 'L'):continue
    print(tc_id,tc_name,tc_time, tc_lat, tc_lon)  
    fix_time = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(tc_time.year,tc_time.month,tc_time.day,tc_time.hour,tc_time.minute)
            
    
    found_overpass = False
    mindist=9999.
    file_list = glob.glob(f'{data_dir}/SSMIS/1C.*')
    file_list.sort()
    for file in file_list:
        #print(file)
        infile = file.split('.')
        satid = infile[1]
        
        overpass_time = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][10:12]),int(infile[4][12:14]))
        
        if(abs(tc_time-overpass_time) < datetime.timedelta(hours=12)):
            
            h5id = Dataset(file)
            #print(h5id.groups)
        
            s1lat = np.array(h5id.groups['S1'].variables['Latitude'])
            s1lon = np.array(h5id.groups['S1'].variables['Longitude'])
            #s1lon[s1lon > 0] = 1e-3*s1lon[s1lon > 0]-179
            #s1lon_unfolded = s1lon
            #s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360.
            #print(s1lat.shape)
            dist = 6371.*np.arccos(np.sin(np.deg2rad(s1lat[:,45]))*np.sin(np.deg2rad(tc_lat)) + np.cos(np.deg2rad(s1lat[:,45]))*np.cos(np.deg2rad(tc_lat))*np.cos(np.deg2rad(s1lon[:,45])-np.deg2rad(tc_lon)))
            if(np.min(dist) > 800): 
                h5id.close()
                continue
            #print(dist.shape)
            #print(satid,abs(overpass_time-tc_time),np.min(dist))
            #if(np.min(dist) < mindist):
            mindist = np.min(dist)
            mintime = np.argmin(dist)
            #print(mindist, mintime)
            #print(np.unravel_index(mintime,s1lat.shape))
            s1hour = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Hour'])
            s1min = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Minute'])
            s1sec = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Second'])
            s1mon = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Month'])
            s1dom = np.array(h5id.groups['S1'].groups['ScanTime'].variables['DayOfMonth'])
            s1year = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Year'])
            #print(s1year.shape)
            mintimestamp = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(s1year[mintime],s1mon[mintime],s1dom[mintime],s1hour[mintime],s1min[mintime])
            
            s1Tc = np.array(h5id.groups['S1'].variables['Tc'])
            
            s2lat = np.array(h5id.groups['S2'].variables['Latitude'])
            s2lon = np.array(h5id.groups['S2'].variables['Longitude'])
            s2Tc = np.array(h5id.groups['S2'].variables['Tc'])
            
            s3lat = np.array(h5id.groups['S3'].variables['Latitude'])
            s3lon = np.array(h5id.groups['S3'].variables['Longitude'])
            s3Tc = np.array(h5id.groups['S3'].variables['Tc'])
            
            s4lat = np.array(h5id.groups['S4'].variables['Latitude'])
            s4lon = np.array(h5id.groups['S4'].variables['Longitude'])
            s4Tc = np.array(h5id.groups['S4'].variables['Tc'])
            
            
            #print(s1Tc.shape)
            h5id.close()
            
            #find min and max scan for plotting
            a = np.where((np.abs(s1lon-tc_lon) < 6) & (np.abs(s1lat-tc_lat) < 6))
            #print(len(a[0]))
            
            if(len(a[0]) > 0):
                s1min = np.min(a[0])#-1
                s1max = np.max(a[0])#+1
                
                s1lon = s1lon[s1min:s1max,:]
                s1lat = s1lat[s1min:s1max,:]
                s1Tc = s1Tc[s1min:s1max,:,:]
                
                s2lon = s2lon[s1min:s1max,:]
                s2lat = s2lat[s1min:s1max,:]
                s2Tc = s2Tc[s1min:s1max,:,:]
                
                s3lon = s3lon[s1min:s1max,:]
                s3lat = s3lat[s1min:s1max,:]
                s3Tc = s3Tc[s1min:s1max,:,:]
                
                s4lon = s4lon[s1min:s1max,:]
                s4lat = s4lat[s1min:s1max,:]
                s4Tc = s4Tc[s1min:s1max,:,:]
                
               # if(tc_lon > )
                s1lon_unfolded = np.copy(s1lon)
                s1lon_unfolded[s1lon < 0] += 360
                s2lon_unfolded = np.copy(s2lon)
                s2lon_unfolded[s2lon < 0] += 360
                s3lon_unfolded = np.copy(s3lon)
                s3lon_unfolded[s3lon < 0] += 360
                s4lon_unfolded = np.copy(s4lon)
                s4lon_unfolded[s4lon < 0] += 360
                
                
            s1ch = ['19V','19H','22V']
            s2ch = ['37V','37H']
            s4ch = ['91V','91H']
            s3ch = ['150H','183$\pm$1H','183$\pm$3H','183$\pm$6H']
            
            #create lat/lon grid for interpolation
            gridlats = np.linspace(tc_lat-rs,tc_lat+rs,num=40*rs+1)
            gridlons = np.linspace(tc_lon-rs,tc_lon+rs,num=40*rs+1)
            #gridlons[gridlons < -180] = gridlons[gridlons < -180] + 360
            #gridlons[gridlons > 180] = gridlons[gridlons > 180] - 360
            
            #Plot native resolution Tbs
            for c in range(0,2):
                m = plt.subplot(2,6,1+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),vmin=100,vmax=300, transform = ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s1ch[c])
            
                m = plt.subplot(2,6,3+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s2lon_unfolded,s2lat,s2Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),vmin=100,vmax=300, transform = ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s2ch[c])
                
                m = plt.subplot(2,6,4+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s4lon_unfolded,s4lat,s4Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),vmin=100,vmax=300, transform = ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s4ch[c])
                
                m = plt.subplot(2,6,5+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s3lon_unfolded,s3lat,s3Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),vmin=100,vmax=300, transform = ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s3ch[c])
                
                m = plt.subplot(2,6,6+6*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s3lon_unfolded,s3lat,s3Tc[:,:,c+2],cmap=tol_cmap(colormap='rainbow_WhBr'),vmin=100,vmax=300, transform = ccrs.PlateCarree())
                plt.colorbar(cm,orientation='horizontal')
                plt.title(s3ch[c+2])
                
            m = plt.subplot(2,6,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,2],cmap=tol_cmap(colormap='rainbow_WhBr'),vmin=50,vmax=300, transform = ccrs.PlateCarree())
            plt.colorbar(cm,orientation='horizontal')
            plt.title(s1ch[2])
            
            plt.show()
            
            
            s1lon_unfolded = np.copy(s1lon)
            if(np.min(gridlons) < -180):
                s1lon_unfolded[s1lon > 0] = s1lon[s1lon > 0]-360
            elif(np.max(gridlons) > 180):
                s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360
            

            xy = np.zeros((s1lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s1lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s1lat)
            z = np.ndarray.flatten(s1Tc[:,:,2])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,0])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,1])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.

            m = plt.subplot(1,3,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '19 + 22 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            #m.coastlines('50m', linewidth=0.8,color='black')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' SSMIS Overpass time: '+mintimestamp)

            s2lon_unfolded = np.copy(s2lon)
            s4lon_unfolded = np.copy(s4lon)
            if(np.min(gridlons) < -180):
                s2lon_unfolded[s2lon > 0] = s2lon[s2lon > 0]-360
                s4lon_unfolded[s4lon > 0] = s4lon[s4lon > 0]-360
            elif(np.max(gridlons) > 180):
                s2lon_unfolded[s2lon < 0] = s2lon[s2lon < 0]+360
                s4lon_unfolded[s4lon < 0] = s4lon[s4lon < 0]+360
            
            xy = np.zeros((s4lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s4lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s4lat)
            z = np.ndarray.flatten(s4Tc[:,:,0])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s4Tc[:,:,1])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            xy = np.zeros((s2lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s2lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s2lat)
            z = np.ndarray.flatten(s2Tc[:,:,1])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.

            m = plt.subplot(1,3,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '37 + 89 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            #m.coastlines('50m', linewidth=0.8,color='black')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' SSMIS Overpass time: '+mintimestamp)

            s3lon_unfolded = np.copy(s3lon)
            if(np.min(gridlons) < -180):
                s3lon_unfolded[s3lon > 0] = s3lon[s3lon > 0]-360
            elif(np.max(gridlons) > 180):
                s3lon_unfolded[s3lon < 0] = s3lon[s3lon < 0]+360
            
            xy = np.zeros((s3lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s3lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s3lat)
            z = np.ndarray.flatten(s3Tc[:,:,3])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s3Tc[:,:,2])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s3Tc[:,:,1])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            maxt = np.max(s3Tc[:,:,1:4])
            mint = np.min(s3Tc[:,:,1:4])
            if(maxt > 300): maxt = 300
            if(mint < 100): mint = 100
            
            idata[:,:,0] = (maxt-rgrid)/(maxt-mint)
            idata[:,:,1] = (maxt-ggrid)/(maxt-mint)
            idata[:,:,2] = (maxt-bgrid)/(maxt-mint)
            idata[idata < 0] = 0
            idata[idata > 1] = 1

            m = plt.subplot(1,3,3, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '183 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            m.coastlines('50m', linewidth=0.8,color='black')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' SSMIS Overpass time: '+mintimestamp)

            
            plt.show()
            
    

In [ ]:
#Download last 12 hours of GMI and DPR data
flist = subprocess.check_output(['curl','-k', '-4','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/GMI/'],text=True)
#print(flist)
flist = flist.split('\n')
for fitem in flist[0:-1]:
    #print(fitem)
    file = fitem.split()[-1]
    #print(file)#,fsize)
    #only run if file is newer than end time from last 1C production file
    if((file[0:2] == '1C')):
        fsize = int(fitem.split()[4])
        fname = file.split('.')
        this_datetime = datetime.datetime(int(fname[4][0:4]),int(fname[4][4:6]),int(fname[4][6:8]),int(fname[4][18:20]),int(fname[4][20:22]),int(fname[4][22:24]))
        if(this_datetime > (datetime.datetime.utcnow() - timedelta(hours=12))):
            dname = (f'{data_dir}/GMI/{file}')
            dsize=-1
            if(os.path.isfile(dname)): dsize = os.path.getsize(dname)
            print(dname,dsize,fsize)
            if(fsize != dsize): call(['curl','-4','-k','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/1C/GMI/'+file,'-o',dname])

#clean GMI directory
files = glob.glob(f'{data_dir}/GMI/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400): os.remove(file)

#Move this to plotting - only download DPR data if good overpass found
flist = subprocess.check_output(['curl','-k', '-4','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/radar/DprL2/'],text=True)
#print(flist)
flist = flist.split('\n')
for fitem in flist[0:-1]:
    #print(fitem)
    file = fitem.split()[-1]
    #print(file)#,fsize)
    #only run if file is newer than end time from last 1C production file
    if((file[0:10] == '2A.GPM.DPR')):
        fsize = int(fitem.split()[4])
        fname = file.split('.')
        this_datetime = datetime.datetime(int(fname[4][0:4]),int(fname[4][4:6]),int(fname[4][6:8]),int(fname[4][18:20]),int(fname[4][20:22]),int(fname[4][22:24]))
        if(this_datetime > (datetime.datetime.utcnow() - timedelta(hours=12))):
            dname = (f'{data_dir}/DPR/{file}')
            dsize=-1
            if(os.path.isfile(dname)): dsize = os.path.getsize(dname)
            print(dname,dsize,fsize)
            if(fsize != dsize): call(['curl','-4','-k','--ftp-ssl-control','--user',f'{pps_user}:{pps_pwd}','ftp://jsimpson.pps.eosdis.nasa.gov/NRTPUB/radar/DprL2/'+file,'-o',dname])

#clean DPR directory
files = glob.glob(f'{data_dir}/DPR/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400*7): os.remove(file)

In [ ]:
#Plot GMI RGB and overlay DPR
proj = ccrs.PlateCarree(central_longitude=180) 
plt.rcParams['figure.figsize'] = [16, 8]
plot_dpr=True
rs= 7
for line in tcvitals:
    #print(str(line).split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    #if(tc_id[2] != 'L'):continue
    print(tc_id,tc_name,tc_time, tc_lat, tc_lon)  
    fix_time = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(tc_time.year,tc_time.month,tc_time.day,tc_time.hour,tc_time.minute)
            
    
    found_overpass = False
    mindist=9999.
    file_list = glob.glob(f'{data_dir}/GMI/1C.*')
    file_list.sort()
    dpr_files = glob.glob(f'{data_dir}/DPR/2A.*')
    dpr_files.sort()
    for n,file in enumerate(file_list):
        if(found_overpass): continue
        #print(file)
        infile = file.split('.')
        satid = infile[1]
        
        gmi_time = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][10:12]),int(infile[4][12:14]))
        
        if(abs(tc_time-gmi_time) < datetime.timedelta(hours=12)):
            #print(file)
            h5id = Dataset(file)
            #print(h5id.groups)
        
            s1lat = np.array(h5id.groups['S1'].variables['Latitude'])
            s1lon = np.array(h5id.groups['S1'].variables['Longitude'])
            #s1lon_unfolded = s1lon
            #s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360.
            #print(s1lat.shape)
            
            dist = 6371.*np.arccos(np.sin(np.deg2rad(s1lat[:,110]))*np.sin(np.deg2rad(tc_lat)) + np.cos(np.deg2rad(s1lat[:,110]))*np.cos(np.deg2rad(tc_lat))*np.cos(np.deg2rad(s1lon[:,110])-np.deg2rad(tc_lon)))
            #print(np.min(dist))
            if(np.min(dist) > 500): 
                h5id.close()
                continue
            
            found_overpass=True
            
            if(np.min(dist) < mindist):
                mindist = np.min(dist)
                mintime = np.argmin(dist)
            #print(np.unravel_index(mintime,s1lat.shape))
            print(file)
            print(mindist,mintime)
            #continue
            s1hour = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Hour'])
            s1min = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Minute'])
            s1sec = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Second'])
            s1mon = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Month'])
            s1dom = np.array(h5id.groups['S1'].groups['ScanTime'].variables['DayOfMonth'])
            s1year = np.array(h5id.groups['S1'].groups['ScanTime'].variables['Year'])

            mintimestamp = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(s1year[mintime],s1mon[mintime],s1dom[mintime],s1hour[mintime],s1min[mintime])
            
            overpass_time = datetime.datetime(s1year[mintime],s1mon[mintime],s1dom[mintime],s1hour[mintime],s1min[mintime],s1sec[mintime])
            print(satid,overpass_time,tc_time,np.min(dist))
            s1Tc = np.array(h5id.groups['S1'].variables['Tc'])
            
            s2lat = np.array(h5id.groups['S2'].variables['Latitude'])
            s2lon = np.array(h5id.groups['S2'].variables['Longitude'])
            s2Tc = np.array(h5id.groups['S2'].variables['Tc'])

            h5id.close()
            #print(file_list[n-1],file_list[n+1])
            #continue
            #open previous and subsequent files and append
            if(n > 0):
                h5id = Dataset(file_list[n-1])
                s1lat2 = np.array(h5id.groups['S1'].variables['Latitude'])
                s1lon2 = np.array(h5id.groups['S1'].variables['Longitude'])
                s1Tc2 = np.array(h5id.groups['S1'].variables['Tc'])
                s2lat2 = np.array(h5id.groups['S2'].variables['Latitude'])
                s2lon2 = np.array(h5id.groups['S2'].variables['Longitude'])
                s2Tc2 = np.array(h5id.groups['S2'].variables['Tc'])
                h5id.close()
                
                s1lat = np.concatenate((s1lat2,s1lat),axis=0)
                s1lon = np.concatenate((s1lon2,s1lon),axis=0)
                s1Tc = np.concatenate((s1Tc2,s1Tc),axis=0)
                s2lat = np.concatenate((s2lat2,s2lat),axis=0)
                s2lon = np.concatenate((s2lon2,s2lon),axis=0)
                s2Tc = np.concatenate((s2Tc2,s2Tc),axis=0)
                
            if(n < len(file_list)-1):
                h5id = Dataset(file_list[n+1])
                s1lat2 = np.array(h5id.groups['S1'].variables['Latitude'])
                s1lon2 = np.array(h5id.groups['S1'].variables['Longitude'])
                s1Tc2 = np.array(h5id.groups['S1'].variables['Tc'])
                s2lat2 = np.array(h5id.groups['S2'].variables['Latitude'])
                s2lon2 = np.array(h5id.groups['S2'].variables['Longitude'])
                s2Tc2 = np.array(h5id.groups['S2'].variables['Tc'])
                h5id.close()
                
                s1lat = np.concatenate((s1lat,s1lat2),axis=0)
                s1lon = np.concatenate((s1lon,s1lon2),axis=0)
                s1Tc = np.concatenate((s1Tc,s1Tc2),axis=0)
                s2lat = np.concatenate((s2lat,s2lat2),axis=0)
                s2lon = np.concatenate((s2lon,s2lon2),axis=0)
                s2Tc = np.concatenate((s2Tc,s2Tc2),axis=0)
                
            s1lon_unfolded = np.copy(s1lon)
            s1lon_unfolded[s1lon < 0] += 360
            s2lon_unfolded = np.copy(s2lon)
            s2lon_unfolded[s2lon < 0] += 360
            #continue
            #find dpr file that corresponds to the overpass time, and read in data
            
            for dpr_file in dpr_files:
                #print(dpr_file)
                infile = dpr_file.split('.')
                #print(infile)
                dpr_start = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][10:12]),int(infile[4][12:14]),int(infile[4][14:16]))
                #print(dpr_start)
                dpr_end = datetime.datetime(int(infile[4][0:4]),int(infile[4][4:6]),int(infile[4][6:8]),int(infile[4][18:20]),int(infile[4][20:22]),int(infile[4][22:24]))
                #print(dpr_end)
                if((dpr_start <= overpass_time) & (dpr_end >= overpass_time)):
                    break
            #print(dpr_file)
            
            if(plot_dpr):
                h5id2 = Dataset(dpr_file)

                nslat = np.array(h5id2.groups['FS'].variables['Latitude'])
                nslon = np.array(h5id2.groups['FS'].variables['Longitude'])
                nsbinst = np.array(h5id2.groups['FS'].groups['PRE'].variables['binStormTop'])
                nsbincfb = np.array(h5id2.groups['FS'].groups['PRE'].variables['binClutterFreeBottom'])
                nshgtst = np.array(h5id2.groups['FS'].groups['PRE'].variables['heightStormTop'])
                nsdbz = np.array(h5id2.groups['FS'].groups['PRE'].variables['zFactorMeasured'])
                nspcp = np.array(h5id2.groups['FS'].groups['SLV'].variables['precipRateESurface'])
                ns_nrcs = np.array(h5id2.groups['FS'].groups['PRE'].variables['sigmaZeroMeasured'])
                h5id2.close()
                print(nsdbz.shape)
                #continue
                max_dbz = np.zeros(nslat.shape,dtype=np.float32)
                for i in range(0,nslat.shape[0]):
                    for j in range(0,nslat.shape[1]):
                        if((nsbinst[i,j] > 0) & (nsbinst[i,j] < nsbincfb[i,j]-1)):
                            #print(i,j,nsbinst[i,j],nsbincfb[i,j])
                            max_dbz[i,j] = np.max(nsdbz[i,j,nsbinst[i,j]:nsbincfb[i,j]-1,0])
            
            
            s1ch = ['10V','10H','18V','18H','23V','36V','36H','89V','89H']
            s2ch = ['166V','166H','183$\pm$3','183$\pm$7']

            #Plot native resolution Tbs
            for c in range(0,2):
                m = plt.subplot(3,5,1+5*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='vertical')
                plt.title(s1ch[c])
            
                m = plt.subplot(3,5,2+5*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c+2],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='vertical')
                plt.title(s1ch[c+2])
                
                m = plt.subplot(3,5,3+5*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c+5],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='vertical')
                plt.title(s1ch[c+5])
                
                m = plt.subplot(3,5,4+5*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,c+7],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='vertical')
                plt.title(s1ch[c+7])
                
                m = plt.subplot(3,5,5+5*c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s2lon_unfolded,s2lat,s2Tc[:,:,c],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='vertical')
                plt.title(s2ch[c])
                
                m = plt.subplot(3,5,13+c, projection=proj)
                m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
                cm = m.pcolormesh(s2lon_unfolded,s2lat,s2Tc[:,:,c+2],cmap=tol_cmap(colormap='rainbow_WhBr'),
                                  vmin=50,vmax=300,transform=ccrs.PlateCarree())
                plt.colorbar(cm,orientation='vertical')
                plt.title(s2ch[c+2])
            
                
            m = plt.subplot(3,5,12, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s1lon_unfolded,s1lat,s1Tc[:,:,4],cmap=tol_cmap(colormap='rainbow_WhBr'),
                              vmin=50,vmax=300,transform=ccrs.PlateCarree())
            plt.colorbar(cm,orientation='vertical')
            plt.title(s1ch[4])
            
            m = plt.subplot(3,5,15, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            cm = m.pcolormesh(s2lon_unfolded,s2lat,s2Tc[:,:,0]-s2Tc[:,:,1],cmap=plt.cm.seismic,
                              vmin=-20,vmax=20,transform=ccrs.PlateCarree())
            plt.colorbar(cm,orientation='vertical')
            plt.title('166V-H')
            
            plt.show()
            
            
            
            #create lat/lon grid for interpolation
            gridlats = np.linspace(tc_lat-rs,tc_lat+rs,num=rs*40+1)
            gridlons = np.linspace(tc_lon-rs,tc_lon+rs,num=rs*40+1)
            #gridlons[gridlons < -180] = gridlons[gridlons < -180] + 360
            #gridlons[gridlons > 180] = gridlons[gridlons > 180] - 360

            s1lon_unfolded = np.copy(s1lon)
            s2lon_unfolded = np.copy(s2lon)
            if(np.min(gridlons) < -180):
                s1lon_unfolded[s1lon > 0] = s1lon[s1lon > 0]-360
                s2lon_unfolded[s2lon > 0] = s2lon[s2lon > 0]-360
            elif(np.max(gridlons) > 180):
                s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360
                s2lon_unfolded[s2lon < 0] = s2lon[s2lon < 0]+360
            
            s1Tc[:,0,:] = 320
            s1Tc[:,-1,:] = 320
            xy = np.zeros((s1lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s1lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s1lat)
            z = np.ndarray.flatten(s1Tc[:,:,2])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,3])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,1])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.

            m = plt.subplot(1,3,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '10 + 18 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.xlabels_top = False
            #m.coastlines('50m', linewidth=0.8,color='black')
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' GMI Overpass time: '+mintimestamp)
            
            idata1 = idata
            
            
            
            z = np.ndarray.flatten(s1Tc[:,:,7])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,8])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s1Tc[:,:,6])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            idata[:,:,0] = 1*(320-rgrid)
            idata[:,:,1] = 1*(320-ggrid)
            idata[:,:,2] = 1*(320-bgrid)
            idata[idata < 0] = 0
            idata[idata > 255] = 255
            idata = idata/255.

            
            m = plt.subplot(1,3,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '36 + 89 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' GMI Overpass time: '+mintimestamp)
        
            idata2 = idata
            
            
            maxt = np.max(s2Tc)
            mint = np.min(s2Tc)
            if(maxt > 300): maxt = 300
            if(mint < 50): mint = 50
            s2Tc[:,0,:] = maxt
            s2Tc[:,-1,:] = maxt
            xy = np.zeros((s2lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s2lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s2lat)
            z = np.ndarray.flatten(s2Tc[:,:,0])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s2Tc[:,:,3])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(s2Tc[:,:,2])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            
            
            idata[:,:,0] = (maxt-rgrid)/(maxt-mint)
            idata[:,:,1] = (maxt-ggrid)/(maxt-mint)
            idata[:,:,2] = (maxt-bgrid)/(maxt-mint)
            idata[idata < 0] = 0
            idata[idata > 1] = 1

            m = plt.subplot(1,3,3, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '166 + 183 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.xlabels_top = False
            m.coastlines('50m', linewidth=0.8,color='white')
            
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' GMI Overpass time: '+mintimestamp)
            plt.show()
            if(plot_dpr==False): continue
            m = plt.subplot(1,3,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata1,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '10 + 18 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            sc = m.scatter(nslon,nslat,c=np.log(nspcp),s=1,vmin=np.log(0.1),vmax=np.log(100),
                           cmap=plt.cm.nipy_spectral,transform=ccrs.PlateCarree())
            cb = plt.colorbar(sc,shrink=0.7,label='Precip rate mm/hr',orientation='horizontal',ticks = [np.log(0.1),np.log(1),np.log(10),np.log(100)])
            cb.ax.set_xticklabels(['0.1','1.0','10','100'])
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.top_labels = False
            gl.right_labels = False
            #m.coastlines('50m', linewidth=0.8,color='black')
            #plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' GMI Overpass time: '+mintimestamp)
            
            m = plt.subplot(1,3,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata2,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '37 + 89 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.xlabels_top = False
            a = max_dbz > 0
            sc = m.scatter(nslon[a],nslat[a],c=max_dbz[a],s=1,vmin=10,vmax=50,cmap=plt.cm.nipy_spectral,
                          transform=ccrs.PlateCarree())
            cb = plt.colorbar(sc,shrink=0.7,label='Composite dBZ',orientation='horizontal')
            gl.xlabels_top = False
            gl.ylabels_right = False
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' GMI Overpass time: '+mintimestamp)
            #m.coastlines('50m', linewidth=0.8,color='black')
            
            
            #second row with DPR overlays: sfc precip rate, max reflectivity, and echo top height
            m = plt.subplot(1,3,3, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            m.text(0.95, 0.01, '166 + 183 GHz Composite',verticalalignment='bottom', horizontalalignment='right',transform=m.transAxes,color='white')
            gl = m.gridlines(color="black", linestyle="dotted",draw_labels=True)
            gl.xlabels_top = False
            m.coastlines('50m', linewidth=0.8,color='white')
            a = nshgtst > 0
            sc = m.scatter(nslon[a],nslat[a],c=1e-3*nshgtst[a],s=1,vmin=0,vmax=20,cmap=plt.cm.nipy_spectral,
                          transform=ccrs.PlateCarree())
            cb = plt.colorbar(sc,shrink=0.7,label='Echo Top (km)',orientation='horizontal')
            gl.top_labels = False
            gl.right_labels = False
            plt.show()
            
            #nrcs plot
            #m = plt.subplot(1,2,1, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            ##a = nshgtst >= 0
            #pcm = m.pcolormesh(nslon,nslat,ns_nrcs[:,:,1],vmin=-5,vmax=15)
            #cb = plt.colorbar(pcm,shrink=0.7,label='Ka-band Measured NRCS',orientation='horizontal')
            #gl.top_labels = False
            #gl.right_labels = False
            
            #m = plt.subplot(1,2,2, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #a = nshgtst <= 0
            #ka_nrcs = ns_nrcs[:,:,1]
            #ka_nrcs = np.ma.masked_where((nshgtst > 0) | (ka_nrcs < -99), ka_nrcs)
            #pcm = m.pcolormesh(nslon,nslat,ka_nrcs,vmin=-5,vmax=15)
            #cb = plt.colorbar(pcm,shrink=0.7,label='Ka-band Measured NRCS (rain-free)',orientation='horizontal')
            #gl.top_labels = False
            #gl.right_labels = False
            #plt.show()
            
            #multi-level dBZ plot
            #pos = [1,2,3,4,5,6,8,9,10,11,12,13, 15,16, 17,18,19,20]
            #for c in range(0,18):
            #    m = plt.subplot(3,7,pos[c],projection=ccrs.PlateCarree())
            #    m.set_extent([tc_lon-4, tc_lon+3, tc_lat-3, tc_lat+4], crs=ccrs.Geodetic())
            #    zdata = nsdbz[:,:,176-(c*4+16),1]
            #    zdata = np.ma.masked_where(zdata < 18, zdata)
            #    pcm = m.pcolormesh(nslon[:,:],nslat[:,:],zdata,vmin=15,vmax=40,
            #                       cmap=plt.cm.nipy_spectral, transform=ccrs.PlateCarree())
            #    gl.top_labels = False
            #    gl.right_labels = False
            #    m.coastlines('50m', linewidth=0.8,color='k')
            #    plt.title('Ka dBZ @ %4.2fkm'%(0.5*c+2))
            #    #ax.xaxis.set_ticklabels([])
            #    #ax.yaxis.set_ticklabels([])
            #m = plt.subplot(3,7,7,projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-4, tc_lon+3, tc_lat-3, tc_lat+4], crs=ccrs.Geodetic())
            #pcm = m.pcolormesh(nslon,nslat,ns_nrcs[:,:,1],vmin=-40,vmax=20)
            ##cb = plt.colorbar(pcm,shrink=0.7,orientation='vertical')
            #plt.title('Ka NRCS (dB)')
            #m = plt.subplot(3,7,14,projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-4, tc_lon+3, tc_lat-3, tc_lat+4], crs=ccrs.Geodetic())
            #max_dbz = np.ma.masked_where(nshgtst <= 0, max_dbz)
            #pcm = m.pcolormesh(nslon,nslat,max_dbz,vmin=15,vmax=40,
            #                   cmap=plt.cm.nipy_spectral, transform=ccrs.PlateCarree())
            #cb = plt.colorbar(pcm,shrink=0.7,orientation='vertical')
            #plt.title('Ka Composite dBZ')
            
            #m = plt.subplot(3,7,21,projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-4, tc_lon+3, tc_lat-3, tc_lat+4], crs=ccrs.Geodetic())
            #nshgtst = np.ma.masked_where(nshgtst <= 0, nshgtst)
            #pcm = m.pcolormesh(nslon,nslat,1e-3*nshgtst,vmin=0,vmax=20,
            #                   cmap=plt.cm.terrain, transform=ccrs.PlateCarree())
            ##cb = plt.colorbar(pcm,shrink=0.7,orientation='vertical')
            #plt.title('Echo top height')
            #plt.show()
            
    

In [ ]:
#Download last 24h of TROPICS Pathfinder data

import shutil


tday = datetime.datetime.utcnow()
yday = tday - datetime.timedelta(hours=24)
yday1 = tday - datetime.timedelta(hours=48)
yday2 = tday - datetime.timedelta(hours=72)

TROPICS_sats = ['TROPICS06BRTTL1B.1.0','TROPICS03BRTTL1B.1.0','TROPICS05BRTTL1B.0.2']

for date in [yday,tday]:#[yday2,yday1,yday,tday]:
    jday = (date - datetime.datetime(tday.year,1,1)).days+1
    for tropics_dir in TROPICS_sats:
        tropics_url = f'https://disc2.gesdisc.eosdis.nasa.gov/data/TROPICS_L1B/{tropics_dir}/{date.year}/'
        dlist = subprocess.check_output(['curl','-k','-4','--ftp-ssl-control','--user',tropics_uid+':'+tropics_pass,
                                     tropics_url]).decode("utf-8")
        dlist = dlist.split('\n')

        for line in dlist:#[35:]:
            try:
                dirname = line.split('a href="')[1].split('"')[0]
            except:
                continue
            if(dirname == f"{jday:03}/"):
                print(dirname)
                flist = subprocess.check_output(['curl','-k','-4','--ftp-ssl-control','--user',tropics_uid+':'+tropics_pass,
                                     tropics_url+dirname]).decode("utf-8")
                flist = flist.split('\n')
                for file in flist:
                    if('.nc"' in file): 
                        fname = file.split('a href="')[1].split('"')[0]
                        print(fname)
                        if(os.path.exists(f'/Users/joemunchak/nobackup/TROPICS/{fname}')): continue
                        #print('curl -n -c /Users/joemunchak/.urs_cookies -b /Users/joemunchak/.urs_cookies -LJO --url', 
                        #      tropics_url+dirname+fname)
                        subprocess.call(['curl','-n','-c','/Users/joemunchak/.urs_cookies','-b',
                                         '/Users/joemunchak/.urs_cookies','-LJO', '--url', tropics_url+dirname+fname])
                        shutil.move(fname, f'/Users/joemunchak/nobackup/TROPICS/{fname}')
#clean MHS directory
files = glob.glob('/Users/joemunchak/nobackup/TROPICS/*')
for file in files:
    st=os.stat(file)
    Age=(time.time()-st.st_mtime)
    if(Age > 86400): os.remove(file)           
    

In [ ]:
#Plot TROPICS + composites
rs=7
proj = ccrs.PlateCarree(central_longitude=180) 
plt.rcParams['figure.figsize'] = [12, 6]
for line in tcvitals:
    #print(str(line).split())
    lineb = str(line).split()
    tc_lat = 0.1*float(lineb[5][:-1])
    if(lineb[5][-1] == 'S'): tc_lat = -tc_lat
    tc_lon = 0.1*float(lineb[6][:-1])
    #print(lineb[6][-1])
    if(lineb[6][-1] == 'W'): tc_lon = -tc_lon
    tc_time = datetime.datetime(int(lineb[3][0:4]),int(lineb[3][4:6]),int(lineb[3][6:8]),int(lineb[4][0:2]),int(lineb[4][2:4]))
    tc_id=lineb[1]
    tc_name=lineb[2]
    #if(tc_id[2] != 'L'):continue
    print(tc_id,tc_name,tc_time, tc_lat, tc_lon)  
    fix_time = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(tc_time.year,tc_time.month,tc_time.day,tc_time.hour,tc_time.minute)
            
    
    found_overpass = False
    mindist=9999.
    file_list = glob.glob(f'{data_dir}/TROPICS/TROPICS??.BRTT.*.nc')
    file_list.sort()
    for file in file_list:
        #print(file)
        infile = file.split('.')
        satid = infile[0].split('/')[-1]
        
        overpass_time = datetime.datetime(int(infile[5][2:6]),int(infile[5][6:8]),int(infile[5][8:10]),
                                          int(infile[5][11:13]),int(infile[5][13:15]))
        #print(overpass_time)
  
        
        if(abs(tc_time-overpass_time) < datetime.timedelta(hours=18)):
            print(file)
            h5id = Dataset(file)
            #print(h5id.groups)
            #stop
            s1lat = np.array(h5id['latitude'][0,:,:])
            s1lon = np.array(h5id['longitude'][0,:,:])
            #s1lon_unfolded = s1lon
            #s1lon_unfolded[s1lon < 0] = s1lon[s1lon < 0]+360.
            #print(s1lat.shape)
            dist = 6371.*np.arccos(np.sin(np.deg2rad(s1lat[:,40]))*np.sin(np.deg2rad(tc_lat)) + 
                np.cos(np.deg2rad(s1lat[:,40]))*np.cos(np.deg2rad(tc_lat))*np.cos(np.deg2rad(s1lon[:,40])-
                                                                                  np.deg2rad(tc_lon)))
            if(np.min(dist) > 500): 
                h5id.close()
                continue
            
            print(satid,abs(overpass_time-tc_time),np.min(dist))
            
            if(np.min(dist) < mindist):
                mindist = np.min(dist)
                mintime = np.argmin(dist)
            #print(np.unravel_index(mintime,s1lat.shape))
            s1hour = np.array(h5id['Hour'])
            s1min = np.array(h5id['Minute'])
            s1sec = np.array(h5id['Second'])
            s1mon = np.array(h5id['Month'])
            s1dom = np.array(h5id['Day'])
            s1year = np.array(h5id['Year'])

            mintimestamp = '%4.4i%2.2i%2.2i/%2.2i%2.2i'%(s1year[mintime],s1mon[mintime],s1dom[mintime],
                                                         s1hour[mintime],s1min[mintime])
            
            
            tTc = np.array(h5id['brightness_temperature'])
            
            tlat = np.array(h5id['latitude'])
            tlon = np.array(h5id['longitude'])
            
            
            h5id.close()
            
            #find min and max scan for plotting
            a = np.where((np.abs(s1lon-tc_lon) < 7) & (np.abs(s1lat-tc_lat) < 7))
            print(len(a[0]))
            
            if(len(a[0]) > 0):
                s4min = np.min(a[0])#-1
                s4max = np.max(a[0])#+1
            
                tlon = tlon[:,s4min:s4max,:]
                tlat = tlat[:,s4min:s4max,:]
                tTc = tTc[:,s4min:s4max,:]
                
                
                tlon_unfolded = np.copy(tlon)
                tlon_unfolded[tlon < 0] += 360
               
            #Plot native resolution Tbs
            ch2band = [0,1,1,1,1,2,2,2,3,3,3,4]
            plt.figure(figsize=(12,12))
            for c in range(0,12):
                ax = plt.subplot(3,4,c+1,projection=proj,
                                 extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs])
                tb = tTc[c,:,:]

                p=ax.pcolormesh(tlon_unfolded[ch2band[c],:,:],tlat[ch2band[c],:,:],tb,
                         cmap=tol_cmap(colormap='rainbow_WhBr'),transform=ccrs.PlateCarree(),vmin=100,vmax=300)#,vmin=tmin,vmax=tmax)
                ax.coastlines()
                plt.colorbar(p,shrink=0.75,orientation='horizontal')#,label='Brightness Temperature (K)')
        
                plt.title(f"Ch {c+1}")
            
            plt.show()
            
            
            #create lat/lon grid for interpolation
            gridlats = np.linspace(tc_lat-rs,tc_lat+rs,num=rs*40+1)
            gridlons = np.linspace(tc_lon-rs,tc_lon+rs,num=rs*40+1)
            #gridlons[gridlons < -180] = gridlons[gridlons < -180] + 360
            #gridlons[gridlons > 180] = gridlons[gridlons > 180] - 360

            s4lat = tlat[3,:,:]
            s4lon = tlon[3,:,:]
            s2lat = tlat[0,:,:]
            s2lon = tlon[0,:,:]
            
            s2lon_unfolded = np.copy(s2lon)
            s4lon_unfolded = np.copy(s4lon)
            if(np.min(gridlons) < -180):
                s2lon_unfolded[s2lon > 0] = s2lon[s2lon > 0]-360
                s4lon_unfolded[s4lon > 0] = s4lon[s4lon > 0]-360
            elif(np.max(gridlons) > 180):
                s2lon_unfolded[s2lon < 0] = s2lon[s2lon < 0]+360
                s4lon_unfolded[s4lon < 0] = s4lon[s4lon < 0]+360
            
            xy = np.zeros((s4lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s4lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s4lat)
            z = np.ndarray.flatten(tTc[10,:,:])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(tTc[9,:,:])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            z = np.ndarray.flatten(tTc[8,:,:])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            maxt = 290#np.max(tTc[8:12,:,:])
            mint = 140#np.min(tTc[8:12,:,:])
            idata[:,:,0] = (maxt-rgrid)/(maxt-mint)
            idata[:,:,1] = (maxt-ggrid)/(maxt-mint)
            idata[:,:,2] = (maxt-bgrid)/(maxt-mint)
            idata[idata < 0] = 0
            idata[idata > 1] = 1
            #idata = idata/255.
            
            #m = plt.subplot(2,s1Tc.shape[2],1, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,rgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],2, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,ggrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            
            #m = plt.subplot(2,s1Tc.shape[2],3, projection=ccrs.PlateCarree())
            #m.set_extent([tc_lon-5, tc_lon+5, tc_lat-5, tc_lat+5], crs=ccrs.Geodetic())
            #cm = m.pcolormesh(gridlons,gridlats,bgrid,cmap=plt.cm.nipy_spectral,vmin=100,vmax=300)
            #plt.colorbar(cm,orientation='horizontal')
            #plt.show()
            m = plt.subplot(1,2,2, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            gl = m.gridlines(color="white", linestyle="dotted",draw_labels=True,alpha=0.3)
            gl.top_labels = False
            m.coastlines('50m', linewidth=0.8,color='white',alpha=0.3)
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+'; \n'+satid+' Overpass time: '+mintimestamp+
                      '\n Water Vapor Composite')
            
            
            
            xy = np.zeros((s2lat.size,2),dtype = np.float32)
            xy[:,0] = np.ndarray.flatten(s2lon_unfolded)
            xy[:,1] = np.ndarray.flatten(s2lat)
            z = np.ndarray.flatten(tTc[0,:,:])
            bgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            z = np.ndarray.flatten(tTc[1,:,:])
            ggrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            z = np.ndarray.flatten(tTc[11,:,:])
            rgrid = interpolate.griddata(xy,z,(gridlons[None,:],gridlats[:,None]), method=interp_method)
            
            idata= np.zeros((rgrid.shape[0],rgrid.shape[1],3),dtype=np.float32)
            maxt = 300#np.max(tTc[1:6,:,:])
            mint = 150#np.min(tTc[1:6,:,:])
            idata[:,:,0] = (maxt-rgrid)/(maxt-mint)
            idata[:,:,1] = (maxt-ggrid)/(maxt-mint)
            mint = 125#np.min(tTc[1:6,:,:])
            idata[:,:,2] = (maxt-bgrid)/(maxt-mint)
            idata[idata < 0] = 0
            idata[idata > 1] = 1
            
            m = plt.subplot(1,2,1, projection=proj)
            m.set_extent([tc_lon-rs, tc_lon+rs, tc_lat-rs, tc_lat+rs], crs=ccrs.Geodetic())
            m.imshow(idata,origin='lower',extent=[tc_lon-rs,tc_lon+rs,tc_lat-rs,tc_lat+rs], transform=ccrs.PlateCarree())
            gl = m.gridlines(color="white", linestyle="dotted",draw_labels=True,alpha=0.3)
            gl.top_labels = False
            m.coastlines('50m', linewidth=0.8,color='white',alpha=0.3)
            plt.title(tc_id+' '+tc_name+' Fix Time: '+ fix_time+';\n '+satid+' Overpass time: '+mintimestamp
                     +'\n Window Channel Composite')
          
            plt.show()

    

In [ ]:
#age of data plots
sensors = ['MHS','ATMS','SSMIS','GMI','AMSR2','TROPICS']
#sensors = ['TMS']
#sensors = ['GMI']
#sensors = ['DPR']
most_recent_second = np.zeros((361,181),dtype=np.float32)
most_recent_source = np.zeros((361,181),dtype=np.int16)
n_overpasses = np.zeros((361,181),dtype=np.int16)
lon_grid = np.linspace(-180,181,num=362)
lat_grid = np.linspace(-90,91,num=182)

today = datetime.datetime.utcnow()

for isensor,sensor in enumerate(sensors):
    print(sensor)
    if(sensor == 'TROPICS'):
        sfiles = glob.glob(f'{data_dir}/{sensor}/TROPICS??.*.ST{today.year:04}{today.month:02}{today.day:02}*.nc')
    else:
        sfiles = glob.glob(f'{data_dir}/{sensor}/1C.*.{today.year:04}{today.month:02}{today.day:02}-S*.RT-H5')
    #sfiles = glob.glob(f'/Users/joemunchak/nobackup/{sensor}/2A.*.RT-H5')
    #sfiles = glob.glob(f'/Users/joemunchak/nobackup/{sensor}/1C.*.RT-H5')
    
    sfiles.sort()
    sensor_overpasses = np.zeros((361,181),dtype=np.int16)
    prev_overpasses = np.zeros((361,181),dtype=np.int16)
    sname = ''
    for sfile in sfiles:
        
        #print(sfile)
        #print(sfile.split('.')[1])
        #if(sname != sfile.split('.')[1]):
        #    print(sname, sfile.split('.')[1])
        n_overpasses += sensor_overpasses
        sensor_overpasses = np.zeros((361,181),dtype=np.int16)
        sname = sfile.split('.')[1]
        ncid = Dataset(sfile)
        if(sensor == 'DPR'):
            slat = np.array(ncid['FS/Latitude'])
            slon = np.array(ncid['FS/Longitude'])
            day = np.array(ncid['FS/ScanTime/DayOfMonth'])
            month = np.array(ncid['FS/ScanTime/Month'])
            year = np.array(ncid['FS/ScanTime/Year'])
            sod = np.array(ncid['FS/ScanTime/SecondOfDay'])
        elif(sensor == 'TROPICS'):
            slat = np.array(ncid['latitude'][0,:,:])
            slon = np.array(ncid['longitude'][0,:,:])
            sod = np.array(ncid['Second'])+60.0*np.array(ncid['Minute'])+3600.0*np.array(ncid['Hour'])
            month = np.array(ncid['Month'])
            day = np.array(ncid['Day'])
            year = np.array(ncid['Year'])
        else:
            slat = np.array(ncid['S1/Latitude'])
            slon = np.array(ncid['S1/Longitude'])
            sod = np.array(ncid['S1/ScanTime/SecondOfDay'])
            day = np.array(ncid['S1/ScanTime/DayOfMonth'])
            month = np.array(ncid['S1/ScanTime/Month'])
            year = np.array(ncid['S1/ScanTime/Year'])
        
        ncid.close()
        
        nscans = len(sod)
        #print(nscans)
        for i in range(0,nscans):
            x = np.mod(np.digitize(slon[i,:],lon_grid)-1,360)
            y = np.digitize(slat[i,:],lat_grid)
            
            for j in range(0,len(x)):
                if(y[j] > 180): 
                    continue
                if(prev_overpasses[x[j],y[j]] == 0): sensor_overpasses[x[j],y[j]] = 1
                soy = sod[i] + (datetime.datetime(year[i],month[i],day[i])-
                                datetime.datetime(year[i],1,1)).total_seconds()
                if(soy > most_recent_second[x[j],y[j]]):
                    most_recent_second[x[j],y[j]] = soy
                    most_recent_source[x[j],y[j]] = isensor
        n_overpasses += sensor_overpasses
        prev_overpasses = sensor_overpasses
        
import datetime




In [ ]:

#sod_now = (today-datetime.datetime(today.year,today.month,today.day)).total_seconds()
sod_now = (today-datetime.datetime(today.year,1,1)).total_seconds()
print(sod_now)
plt.figure(figsize = (16,16))
ax = plt.subplot(2,1,1,projection=ccrs.PlateCarree())
cm=ax.pcolormesh(lon_grid[:-1],lat_grid[10:-10],(sod_now-np.transpose(most_recent_second[:-1,10:-10]))/60,
                 vmin=0,vmax=300,cmap=plt.cm.RdYlGn_r)
                 #vmin=0,vmax=300,cmap=plt.cm.viridis)
plt.colorbar(cm,label='minutes')
plt.title('Time since most recent data collected')
ax.coastlines()

ax = plt.subplot(2,1,2,projection=ccrs.PlateCarree())
cm=ax.pcolormesh(lon_grid[:-1],lat_grid[10:-10],np.transpose(most_recent_source[:-1,10:-10]),
                 vmin=0,vmax=6,cmap=plt.cm.jet)
cbar = plt.colorbar(cm,label='sensor')
cbar.set_ticks(np.arange(len(sensors)))
cbar.set_ticklabels(sensors)
#cm=ax.pcolormesh(lon_grid[:-1],lat_grid[10:-10],np.transpose(n_overpasses[:-1,10:-10]),
#                 vmin=0,cmap=plt.cm.jet)
#plt.colorbar(cm,label='number of overpasses')
ax.coastlines()
plt.title('Sensor providing most recent observation')
plt.show()



### 